In [ ]:
from utils import *
# BEST_MODEL_DIR = BEST_MODEL_DIR
# BEST_MODEL_DIR = 'data/2637759/'
# BEST_MODEL_DIR = 'data/2637760/'
# BEST_MODEL_DIR = 'data/2637771/'
# BEST_MODEL_DIR = 'data/2637772/'

net_path = model_path(FX, BEST_MODEL_DIR)

N_PLOTS = 10 # number of plots to generate, actually a multiplier

ds = LiuqeDataset(EVAL_DS_PATH)

x_mean_std = to_tensor(np.load(f'{BEST_MODEL_DIR}/x_mean_std.npz')['x_mean_std']) # load normalization parameters

m = FullNet(InputNet(x_mean_std), PtsEncoder(), FHead(3), FHead(1), LCFSHead())
m.load_state_dict(torch.load(net_path, map_location=torch.device("cpu"))) # load pretrained model

lcfs = LCFSNet(m.input_net, m.lcfs_head) # initialize LCFS model

for i in range(N_PLOTS): plot_network_outputs(ds, m, title=f'TEST {i}') # plot network outputs

for i in range(N_PLOTS): plot_lcfs_net_out(ds, lcfs, title=f'TEST {i}') # plot LCFS outputs

In [ ]:
# convert model to onnx
rt_net = LiuqeRTNet(m.input_net, m.pts_enc, m.rt_head)
rt_net.eval()
# convert_to_onnx_static(rt_net, save_dir=[BEST_MODEL_DIR, SAVE_DIR])
convert_to_onnx_dyn(rt_net, save_dir=[BEST_MODEL_DIR, SAVE_DIR])

In [ ]:
# test the model on the demo dataset
d = np.load(f'{DS_DIR}/demo.npz')
n_examples = d[PTS].shape[0]
n_pts = d[PTS].shape[1]
n_ctrl_pts = 25

rand_i = 0

phys = d[PHYS][rand_i]
r = d[PTS][rand_i, :n_ctrl_pts, 0]
z = d[PTS][rand_i, :n_ctrl_pts, 1]

pyhs, r, z = to_tensor(phys), to_tensor(r), to_tensor(z)
rt = rt_net(pyhs, r, z)

Fx_pred, Br_pred, Bz_pred = rt[:,0].detach().cpu().numpy(), rt[:, 1].detach().cpu().numpy(), rt[:, 2].detach().cpu().numpy()
Fx_true, Br_true, Bz_true = d[FX][rand_i], d[BR][rand_i], d[BZ][rand_i]

k = 8
np.set_printoptions(precision=4, suppress=True, linewidth=1000)
print(f'Fx_pred -> {Fx_pred[:k]}\nFx_true -> {Fx_true[:k]}')
print(f'Br_pred -> {Br_pred[:k]}\nBr_true -> {Br_true[:k]}')
print(f'Bz_pred -> {Bz_pred[:k]}\nBz_true -> {Bz_true[:k]}')